# Actividad 9B


El objetivo de esta actividad es aplicar los conceptos teóricos vistos en los materiales presentados en la actividad 9A.

Para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones), recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles. Para ello usaremos la siguiente constante.

In [ ]:
RANDOM_STATE = 1234

__Nombre: XX__


### Carga del conjunto de datos

El fichero `IMBD_Dataset.csv` contiene opiniones de películas clasificadas en 2 categorías diferentes (positiva/negativa).

Este set de datos se creó utilizando el "IMDB Dataset of 50K Movie Reviews", el cual contiene 50,000 reseñas de películas con un sentimiento positivo o negativo adjunto a ellas.

In [1]:
# acceso a google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np


In [3]:

path = "/content/drive/MyDrive/Colab Notebooks/EALC/Semana 9/"

imbd_file = path + "IMDB_Dataset.csv"

df = pd.read_csv(imbd_file)
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### TAREAS A REALIZAR

Muestra un ejemplo de cada clase.

Haz un estudio del conjunto de datos. ¿qué palabras aparecen más veces?

Indica porque es necesario normalizar el corpus y normalízalo.

Crea una partición de los datos dejando el 80% para entrenamiento y el 20% restante para test usando la función `train_test_split` de sklearn.

Elige dos modelos de aprendizaje automático y realiza los siguientes pasos: explicación breve de su funcionamiento, preprocesamiento, entrenamiento y evaluación.

Indica cual de los 2 modelos funciona mejor y trata de explicar porqué.

Para el mejor modelo, indica un ejemplo mal clasificado de cada clase y trata de justificar el error.


In [ ]:
# Mostrar un ejemplo de cada clase
ejemplo_negativo = df[df['sentiment'] == 'negative'].sample(1)
print("Ejemplo negativo:", ejemplo_negativo)
print()
ejemplo_positivo = df[df['sentiment'] == 'positive'].sample(1)
print("Ejemplo positivo:", ejemplo_positivo)

Ejemplo negativo:                                                  review sentiment
8463  Well, it is hard to add comment after reading ...  negative

Ejemplo positivo:                                                  review sentiment
4355  While a bit preachy on the topic of progress a...  positive


In [4]:
# ¿Qué palabras aparecen más?

# Vamos primero a estudiar las palabras con mayor frecuencia teniendo en cuenta la totalidad del dataset, sin quitar stopwords
from sklearn.feature_extraction.text import CountVectorizer

textos = df['review'].tolist()

# Inicializar el CountVectorizer
vectorizer = CountVectorizer()

# Transformamos los textos en una matriz de recuentos de palabras
matriz_X = vectorizer.fit_transform(textos)

# Obtenemos las palabras del vectorizador
palabras = vectorizer.get_feature_names_out()

# Sumamos el recuento de cada palabra en todo el conjunto de datos
recuento_palabras = matriz_X.sum(axis=0)

# Creamos un DataFrame para visualizar los resultados
df_recuento_palabras = pd.DataFrame(recuento_palabras, columns=palabras)

# Mostramos las palabras más comunes
palabras_mas_comunes = df_recuento_palabras.T.sort_values(by=0, ascending=False)
print("Palabras más comunes:")
print(palabras_mas_comunes.head(10))

Palabras más comunes:
           0
the   667993
and   324441
of    289410
to    268124
is    211082
br    201951
it    190857
in    186781
this  151002
that  143879


In [5]:
# Normalizar el corpus es necesario, por ejemplo, para eliminar las palabras vacías que no aportan contenido
# semántico a nuestros textos, que no caracterizan los textos y que "contaminan" los resultados de las frecuencias de palabras,
# ya que son las palabras más usadas.

import re
import nltk
from nltk.corpus import stopwords

wpt = nltk.WordPunctTokenizer()
nltk.download('stopwords') # añadido
sw = ['br']
stop_words = set(nltk.corpus.stopwords.words('english')+ sw)

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# Crear una nueva columna 'review_normalized' en el DataFrame
df['review_normalized'] = df['review'].apply(normalize_document)

# Verificar el DataFrame resultante
df.head(10)

,review,sentiment,review_normalized
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode yo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...
5,"Probably my all-time favorite movie, a story o...",positive,probably alltime favorite movie story selfless...
6,I sure would like to see a resurrection of a u...,positive,sure would like see resurrection dated seahunt...
7,"This show was an amazing, fresh & innovative i...",negative,show amazing fresh innovative idea first aired...
8,Encouraged by the positive comments about this...,negative,encouraged positive comments film looking forw...
9,If you like original gut wrenching laughter yo...,positive,like original gut wrenching laughter like movi...


In [10]:
# vuelvo a hacer la lista de palabras más frecuentes sobre el texto normalizado
# ahora no he usado CountVectorizer sino un tokenizador

import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Tokenizar las reseñas
tokenized_reviews = [word_tokenize(review.lower()) for review in df['review_normalized']]

# Unir listas de palabras en una sola lista
all_words = [word for sublist in tokenized_reviews for word in sublist]

# Calcular la frecuencia de cada palabra
fdist = FreqDist(all_words)

# Crear un DataFrame con las palabras y sus frecuencias
df_frecuencia = pd.DataFrame({'Palabra': list(fdist.keys()), 'Frecuencia': list(fdist.values())})

# Ordenar el DataFrame por frecuencia en orden descendente
df_frecuencia = df_frecuencia.sort_values(by='Frecuencia', ascending=False)

# Mostrar las palabras más frecuentes
print("Palabras más frecuentes:")
print(df_frecuencia.head(10))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Palabras más frecuentes:
    Palabra  Frecuencia
292   movie       83536
304    film       74478
0       one       51034
312    like       39001
393    good       28575
230    even       24578
79    would       24028
215    time       23276
184  really       22951
167     see       22536


De esta manera, una vez normalizado el corpus, obtenemos una lista de las palabras más frecuentes en la que todas las palabras que la conforman son palabras léxicas, ya no aparecen palabras vacías, ni signos de puntuación. Las palabras más frecuentes son:


1.   movie
2.   film
3.   one
4.   like
5.   good
6.   even
7.   would
8.   time
9.   really
10.  see



In [11]:
# dividir el corpus en un 80% para el entrenamiento y un 20% para la validación

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

X = df['review_normalized']
y = df['sentiment']

# Dividimos el conjunto de datos en entrenamiento (80 %) y prueba (20 %)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# De esta manera, a lo que en el ejemplo le llaman .data se convierte en X y lo que es .target se convierte en Y

# Imprimir la cantidad de textos en entrenamiento y prueba después de la división
print("Training texts:", len(X_train))
print("Test texts:", len(X_test))

Training texts: 40000
Test texts: 10000


### Preprocesamiento

In [ ]:
# Elige dos modelos de aprendizaje automático y realiza los siguientes pasos: explicación breve de su funcionamiento, preprocesamiento, entrenamiento y evaluación.

# Voy a elegir los modelos Support Vector Machines y Logistic Regression

# Support Vector Machines (svm)
# es un modelo que se usa tanto para regresión como para clasifiación
# se basa en encontrar un hiperplano en el espacio de características que
# mejor separe las clases de datos, maximizando el margen entre las clases.

# Modelo de regresión logística
# se usa para problemas de clasificación
# usa la función logística para transformar la salida lineal a valores entre 0 y 1.
# la salida del modelo es una probabilidad de pertenencia a una clase
# toma una decisión de clasificación basada en esa probabilidad

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

preprocessing = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer())
])

train_preprocessed = preprocessing.fit_transform(X_train)

test_preprocessed = preprocessing.transform(X_test)

### Entrenamiento

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

svm_classifier = LinearSVC()
lr_classifier = LogisticRegression(multi_class="ovr")

print("Training SVM classifier...")
svm_classifier.fit(train_preprocessed, y_train)

print("Training Logistic Regression classifier...")
lr_classifier.fit(train_preprocessed, y_train)

Training SVM classifier...
Training Logistic Regression classifier...


LogisticRegression(multi_class='ovr')

### Evaluación

In [14]:
svm_predictions = svm_classifier.predict(test_preprocessed)
lr_predictions = lr_classifier.predict(test_preprocessed)

In [15]:
import numpy as np

print("SVM Accuracy:", np.mean(svm_predictions == y_test))
print("LR Accuracy:", np.mean(lr_predictions == y_test))

SVM Accuracy: 0.8931
LR Accuracy: 0.8904


En mi opinión, con estos resultados podemos decir que el funcionamiento de ambos modelos es muy parecido, con un porcentaje de acierto bastante elevado y prácticamente igual: 0,89. Igualmente, el que parece que tiene más precisión es el modelo SVM, por eso lo he elegido como el modelo para analizar sus errores.

In [17]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, svm_predictions))

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      4988
    positive       0.89      0.90      0.89      5012

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [18]:
etiquetas_incorrectas = X_test[svm_predictions != y_test]
etiquetas_correctas = y_test[svm_predictions != y_test]
predicciones = svm_predictions[svm_predictions != y_test]

In [19]:
for i in range(5):
    print(f'Ejemplo #{i+1}:')
    print(f'Texto: {etiquetas_incorrectas.iloc[i]}')
    print(f'Etiqueta real: {etiquetas_correctas.iloc[i]} - Predicción: {predicciones[i]}')
    print('\n')

Ejemplo #1:
Texto: grandmother took sister see movie came theaters back happily bought tickets popcorn soda walked right theater sat watch movie audience didnt applauded strongly remember heard people say didnt like didnt like thought rather stupid worth seeing eddie murphy hysterical apart whole movie bad rarely laughed parts also remembered people theater almost hardly even laughed really thought bad making animals talk talking animals exist cartoons live action movies totally mutt said apart eddie murphys hysterical twist brings movie worth watching rather stupidbr seen eddie murphy several movies thought funny said funny part movie also seen eddie murphy really great movie adventures pluto nash movie movie would really recommend see apart eddie murphy probably going like especially lot talking animals ill give movie rating stars possible stars
Etiqueta real: negative - Predicción: positive


Ejemplo #2:
Texto: long keep mind production movie copyright ploy intended serious release 


*   En el primer ejemplo, la predicción fue positiva, pero la etiqueta correcta era la negativa. El problema puede ser que parece que el modelo ha tenido en cuenta palabras como "happily" o "applauded" para etiquetarla como positiva. Puede que el modelo no haya interpretado bien las negaciones como por ejemplo "didnt applauded strongly", "remember heard people say didnt like " o "hardly even laughed". A lo mejor habría que mejorar los parámetros, asegurarse que los textos estén correctamente etiquetados, o darle un corpus de entrenamiento más amplio para que mejore sus predicciones.
*   El ejemplo 5 está etiquetado como positivo, pero el modelo lo ha etiquetado como negativo. El fallo en esta predicción puede ser que la persona que hizo la review da la razón a los críticos que critican la película. La clave se encuentra al final cuando dice: "despite reading several bad reviews cheap looking rubber dinosaurs enjoyed return lost worldbr rating stars". Hay que tener en cuenta que hemos eliminado las palabras vacías, por lo que a veces resulta complicado entender bien las frases. Sin embargo, la clave es el "despite bad reviews" y el "enoyed". Una persona puede sacar la conclusión de que, aunque la película tenga sus fallos, le ha gustado. Pero supongo que para el modelo pesan más todas las palabras negativas que ha escrito en la review antes de dar su opinión sobre la película.
*   El ejemplo 3 me parece curioso porque creo que el fallo podría estar en la etiqueta original. El modelo la ha etiquetado como negativa, pero estaba etiquetado originalmente como positiva. Tras leerla y fijarme en las palabras, me parece que sería una review negativa, basándome en las expresiones: "nothing new", "pretty tame tired stuff" o "never changes hair". Por tanto, puede que en este caso nuestro modelo haya funcionado correctamente y sea el etiquetado original el que se esté equivocando.



